In [55]:
import datetime as dt
from modified_dficf_vectorizer import CountVectorizer, TfidfTransformer, DfIcfTransformer
import datetime as dt
import string
from sklearn.model_selection import train_test_split
import collections
import pandas as pd
import re
from stop_words import get_stop_words
import numpy as np
import random
from copy_of_orj_tfidf_vectorizer import CountVectorizer as org_count_vectorizer

In [19]:
def get_files_content(root_folder, file_extension):
    root_folder_name = root_folder.rsplit(FOLDER_SEPERATOR, 1)[1]
    file_list = []
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            if file.endswith("." + file_extension):
                file_path = os.path.join(root, file)
                parent_folder = root.rsplit(FOLDER_SEPERATOR, 1)[1]
                file_name = file.split('.')[0]
                try:
                    with open(file_path, 'r', encoding='utf-8', errors='ignore') as content_file:
                        content = content_file.read()
                        exclude = set(string.punctuation)
                        content = ''.join(ch.replace("\n", "-") for ch in content if ch not in exclude)
                        content = content.replace("--", " ")
                        content = content.replace("-", " ").strip()
                        # NLTK - STOPWORDS
                        # content = " ".join(
                        #     word.strip() for word in content.split() if word not in stopwords.words('english'))
                        # OPEN SOURCE GITHUB - STOP_WORDS
                        content = " ".join(
                            word.strip() for word in content.split() if word not in get_stop_words('english'))
                        content = re.sub(r'\d+', '', content)

                    file_list.append((parent_folder, file_name, content.lower()))
                except Exception as e:
                    print(str(e))
                    print(parent_folder, file)
    return root_folder_name, pd.DataFrame(data=file_list,
                                          columns=['Procedure', 'AccountID', 'Features'])


In [20]:
def extract_elements_from_2Ddict(_2d_dict):


    X_y = []
    for _key, _values in _2d_dict.items():
        for _element in _values:
            X_y.append((_element, _key))

    #np.random.seed(0)
    random.shuffle(X_y)
    df = pd.DataFrame(data=X_y, columns=['Doc', 'Cat'])
    X = df.Doc.values
    y = df.Cat.values
    del X_y
    del _2d_dict
    return X, y, df

In [23]:
FOLDER_SEPERATOR = "/"
__BASE_FOLDER = r'/home/jackalhan/Development/app_data/tf_idf_hiarchical'
__GENERATED_DATA_FOLDER = 'generated_data'
__FOLDER_NAME = 'bbc'
TARGET_PATH = os.path.join(__BASE_FOLDER, __FOLDER_NAME)
tokenize = lambda doc: doc.lower().split(" ")

In [24]:
data_size_rate = 6

In [39]:
start_time = dt.datetime.now()
print('Files are getting ready')
root_folder_name, files_dataframe = get_files_content(TARGET_PATH, 'txt')
print('{} of files are parsed in {} mins.'.format(str(len(files_dataframe)),
                                                  ((dt.datetime.now() - start_time).seconds) / 60))
np.random.seed(0)
data = files_dataframe.groupby(['Procedure'])
len_of_data = len(data)
train_set = collections.OrderedDict()
test_set = collections.OrderedDict()
for _proc, _grouped in data:
    doc_size = len(_grouped.Features)
    print("Data size of {} proc is {}".format(_proc, doc_size))
    print("With adding an data rate {}".format(int(doc_size / data_size_rate)))
    docs = _grouped.Features.transform(np.random.permutation)
    features_train, features_test = train_test_split(docs[:int(doc_size / data_size_rate)], test_size=0.33)
    train_set[_proc] = list()
    # train_set[_proc].extend([x for x in _grouped.Features])
    train_set[_proc].extend([x for x in features_train])
    test_set[_proc] = list()
    test_set[_proc].extend([x for x in features_test])

Files are getting ready
2225 of files are parsed in 0.03333333333333333 mins.
Data size of business proc is 510
With adding an data rate 85
Data size of entertainment proc is 386
With adding an data rate 64
Data size of politics proc is 417
With adding an data rate 69
Data size of sport proc is 511
With adding an data rate 85
Data size of tech proc is 401
With adding an data rate 66


In [40]:
count_vectorizer = CountVectorizer(stop_words='english', stemming_and_lemmatization=True)
# categorical values from the trained count vectorizer.
vocabularies_with_categories_, number_of_categories, dict_for_term_frequencies, dict_for_number_of_documents, \
dict_for_local_category_document_term_count, dict_for_global_category_document_term_count, \
dict_for_global_category_count = count_vectorizer.fit_transform(train_set)
del count_vectorizer

dficf = DfIcfTransformer(vocabularies_with_categories_, number_of_categories, dict_for_term_frequencies,
                         dict_for_number_of_documents,
                         dict_for_local_category_document_term_count, dict_for_global_category_document_term_count,
                         dict_for_global_category_count)

dficf.transform()
original_text_X, original_text_y, original_X_y_df = extract_elements_from_2Ddict(train_set)
del train_set

CountVectorizer for corpus are getting calculated
CATEGORY: business
1 % of data processed for business category
2 % of data processed for business category
3 % of data processed for business category
4 % of data processed for business category
5 % of data processed for business category
6 % of data processed for business category
7 % of data processed for business category
8 % of data processed for business category
9 % of data processed for business category
10 % of data processed for business category
11 % of data processed for business category
12 % of data processed for business category
13 % of data processed for business category
14 % of data processed for business category
15 % of data processed for business category
16 % of data processed for business category
17 % of data processed for business category
18 % of data processed for business category
19 % of data processed for business category
20 % of data processed for business category
21 % of data processed for business cate

86 % of data processed for entertainment category
87 % of data processed for entertainment category
88 % of data processed for entertainment category
89 % of data processed for entertainment category
90 % of data processed for entertainment category
91 % of data processed for entertainment category
92 % of data processed for entertainment category
93 % of data processed for entertainment category
94 % of data processed for entertainment category
95 % of data processed for entertainment category
96 % of data processed for entertainment category
97 % of data processed for entertainment category
98 % of data processed for entertainment category
99 % of data processed for entertainment category
100 % of data processed for entertainment category
CATEGORY: politics
1 % of data processed for politics category
2 % of data processed for politics category
3 % of data processed for politics category
4 % of data processed for politics category
5 % of data processed for politics category
6 % of dat

72 % of data processed for sport category
73 % of data processed for sport category
74 % of data processed for sport category
75 % of data processed for sport category
76 % of data processed for sport category
77 % of data processed for sport category
78 % of data processed for sport category
79 % of data processed for sport category
80 % of data processed for sport category
81 % of data processed for sport category
82 % of data processed for sport category
83 % of data processed for sport category
84 % of data processed for sport category
85 % of data processed for sport category
86 % of data processed for sport category
87 % of data processed for sport category
88 % of data processed for sport category
89 % of data processed for sport category
90 % of data processed for sport category
91 % of data processed for sport category
92 % of data processed for sport category
93 % of data processed for sport category
94 % of data processed for sport category
95 % of data processed for sport c

40 % of data processed for entertainment category
43 % of data processed for entertainment category
45 % of data processed for entertainment category
48 % of data processed for entertainment category
50 % of data processed for entertainment category
52 % of data processed for entertainment category
55 % of data processed for entertainment category
57 % of data processed for entertainment category
60 % of data processed for entertainment category
62 % of data processed for entertainment category
64 % of data processed for entertainment category
67 % of data processed for entertainment category
69 % of data processed for entertainment category
71 % of data processed for entertainment category
74 % of data processed for entertainment category
76 % of data processed for entertainment category
79 % of data processed for entertainment category
81 % of data processed for entertainment category
83 % of data processed for entertainment category
86 % of data processed for entertainment category


In [41]:
original_X_y_df.to_csv(os.path.join(TARGET_PATH, 'train_documents.csv'))

In [58]:
dficf.fit_transform(original_text_X)

SystemError: Parent module '' not loaded, cannot perform relative import

In [ ]:
X_data, y_data, y_data_, sentence_index = dficf.vectors_